In [1]:
import pandas as pd
import mysql.connector as sql
import math

In [2]:
def term(x,y):
    try:
        bond_term = int(x.year) - int(y.year)
    except:
        bond_term = 100
    return bond_term

In [3]:
def Type_term(x):
    if x >= 10:
        return 'Long Term'
    elif x < 10 and x >= 5:
        return  'Mid Term'
    else:
        return 'Short Term'

In [4]:
def assign_rating(x):
    x = float(x)
    if math.isnan(x):
        return 'UN'
    else:
        if x < 0:
            return 'NR'
        elif x<=13.5:
            return "Junk"
        elif x>13.5 and x<=16.5:
            return 'Lower medium grade'
        elif x>16.5 and x<=19.5:
            return 'Upper medium grade'
        elif x>19.5 and x<=22.5:
            return 'High grade'
        else:
            return 'Prime'

In [5]:
def get_all_bonds_in_list(start_date,end_date):
    print('start to get data from {} to {}'.format(start_date,end_date))
    #####Create engine:
    db_connection = sql.connect(host='0.0.0.0', database='bond_db', user='root', password='password')
    ##security query
    security_query = "select * from security_info"
    ##read data
    security_data = pd.read_sql(security_query,con=db_connection)
    security_data = security_data[['SecurityID','Currency','IssueDate','MaturityDate']]
    ##price query
    price_query = "select * from bond_spread where  KeyDate between '{}' and '{}'".format(start_date,end_date)
    #####read bond_data from db:
    final_data = pd.read_sql(price_query,con=db_connection)
    final_data = final_data.merge(security_data, on=['SecurityID'], how='left')
    final_data.dropna(subset=["ZSpread"],inplace=True)
    final_data_1 = final_data[final_data.Currency == 'USD']
    final_data = final_data_1.groupby('SecurityID')
    bonds_list = [final_data.get_group(x) for x in final_data.groups]
    db_connection.close()
    return bonds_list

In [6]:
def merge_left_right(data1,data2):
    temp_data = data1.append(data2)
    return temp_data

def merg_sort(list_data):
    list_length = len(list_data)
    if list_length == 1:
        output = list_data[0][2]
        return output
    else:
        mid_point = math.floor(list_length/2)
        #print(len(list_data[:mid_point]))
        left = list_data[mid_point:]
        right = list_data[:mid_point]
        return merge_left_right(merg_sort(left), merg_sort(right))

In [7]:
silding_windows = [['2018-12-31','2019-02-28'],['2019-03-01','2019-04-30'],
                   ['2019-05-01','2019-06-30'],['2019-07-01','2019-08-31'],
                   ['2019-09-01','2019-10-31'],['2019-11-01','2019-12-31']]

## Cluster data

In [8]:
cluster_data = pd.read_csv('Cluster_group_125.csv')

In [9]:
cluster_data.drop(columns=['Unnamed: 0'],inplace=True)

In [10]:
cluster_data.head()

,SecurityID,2019-02-28,2019-04-30,2019-06-30,2019-08-31,2019-10-31,2019-12-31
0,3051.0,Cluster 0,Cluster 85,Cluster 70,Cluster 82,Cluster 99,Cluster 91
1,3704.0,Cluster 0,Cluster 119,Cluster 0,Cluster 12,Cluster 65,Cluster 87
2,4121.0,Cluster 0,Cluster 52,Cluster 53,Cluster 72,Cluster 6,Cluster 91
3,4130.0,Cluster 0,Cluster 76,Cluster 124,Cluster 89,Cluster 10,Cluster 109
4,4259.0,Cluster 0,Cluster 95,Cluster 28,Cluster 86,Cluster 105,Cluster 76


In [11]:
def get_the_daily_spread_windows(bonds_list):
    bond_spread_list = []
    for bond in bonds_list:
        try:
            bond['G_change']=bond.GSpread.pct_change()
            bond['ModifiedDuration_Plain_change']=bond.ModifiedDuration_Plain.pct_change()
            bond['YieldWorst_change']=bond.YieldWorst.pct_change()
            bond.dropna(subset=['G_change'],inplace = True)
            bond_spread_list.append((len(bond.GSpread.values),bond.SecurityID.iloc[0],bond))
        except:
            pass
        return bond_spread_list

In [12]:
def Average_daily_Gspread_change_cluster(window,cluster_data):
    ####Grab Data###
    bond_list1 = get_all_bonds_in_list(window[0],window[1])
    ####G Change ###
    daily_spread_1 = get_the_daily_spread_windows(bond_list1)
    new_data = merg_sort(daily_spread_1)
    new_data.dropna(inplace=True)
    data1 = new_data.merge(cluster_data[['SecurityID',window[1]]],on=['SecurityID'],how ='left')
    temp = data1.groupby(by=['KeyDate',window[1]],as_index=False).mean()
    data1['Group'] = data1[window[1]] 
    data1.drop(columns=[window[1]],inplace=True)
    temp['Group'] = temp[window[1]]
    end = temp[['Group','KeyDate','G_change','YieldWorst_change','ModifiedDuration_Plain_change']]
    end = end.rename(columns={'G_change':'Cluster_G_change','YieldWorst_change':'Cluster_average_YieldWorst_change','ModifiedDuration_Plain_change':'Cluster_average_ModifiedDuration_Plain_change'})
    return end,data1

## Read Your secuirty data here

In [13]:
db_connection = sql.connect(host='0.0.0.0', database='bond_db', user='root', password='password')
security_query = "select * from security_info"
security_data = pd.read_sql(security_query,con=db_connection)

## END

## Read Your Rating data

In [14]:
db_connection = sql.connect(host='0.0.0.0', database='bond_db', user='root', password='password')
rating_query = "select * from rating"
rating_data = pd.read_sql(rating_query,con=db_connection)

In [15]:
rating_data = rating_data.groupby(by=['SecurityID'],as_index=False).mean()

In [16]:
security_data['Term'] = security_data.apply(lambda x:term(x.MaturityDate,x.IssueDate),axis=1)

In [17]:
security_data['Term'] = security_data['Term'].fillna(100)
security_data['Type of Term'] = security_data['Term'].apply(Type_term)

In [18]:
security_data = security_data.merge(rating_data,on =['SecurityID'],how='left')

In [19]:
security_data['Rating'] = security_data['RatingSP'].apply(assign_rating)

In [20]:
security_data.columns

Index(['SecurityID', 'OfficialName', 'SecurityType', 'SecuritySubType',
       'SecurityBBGType', 'SecurityBBGYellowKey', 'Series', 'SecuritySector',
       'SecurityIndustry', 'SecuritySubIndustry', 'BBGSecuritySector',
       'IssuerParent', 'Obligor', 'Ticker', 'EquityTicker', 'Issuer',
       'ObligorID', 'CountryDomicile', 'CountryRisk', 'CountryIssue',
       'Currency', 'Coupon', 'CouponType', 'CouponFrequency', 'CouponDayCount',
       'CreationDate', 'LastModifiedTime', 'IssueDate', 'PricingDate',
       'MaturityDate', 'IsCallable', 'IsConvertible', 'IsContingent',
       'IsFactorable', 'IsHybrid', 'IsMaple', 'IsPrivatePlacement',
       'IsPreliminary', 'IsPerpetual', 'IsJunior', 'Seniority', 'PaymentRank',
       'IsBailIn', 'QuoteSize', 'AmountIssued', 'MinIncrement', 'MinPiece',
       'BBGDescription', 'Term', 'Type of Term', 'RatingSP', 'Rating'],
      dtype='object')

In [45]:
def get_the_cluster_data(security_info,cluster_data,windows):
    print('start to get data for {}'.format(windows[1]))
    security_info = security_data[['SecurityID','SecuritySector','AmountIssued','Coupon','Seniority','Term','RatingSP','Rating','Type of Term']]
    a = security_info.merge(cluster_data[['SecurityID',windows[1]]],on=['SecurityID'],how='left')
    a.dropna(subset=[windows[1]],inplace=True)
    time_period = a.rename(columns={'2019-02-28':'Group'})

    Group_Seniority = time_period.groupby(by=['Group','Seniority']).count()
    Group_sector = time_period.groupby(by=['Group','SecuritySector']).count()
    Group_Type_term = time_period.groupby(by=['Group','Type of Term']).count()
    Group_Rating = time_period.groupby(by=['Group','Rating']).count()
    
    Group = time_period.groupby(by=['Group']).count()
    Sector = time_period.groupby(by=['SecuritySector']).count()
    Type_of_Term = time_period.groupby(by=['Type of Term']).count()
    Seniority = time_period.groupby(by=['Seniority']).count()
    Rating = time_period.groupby(by=['Rating']).count()
    
    Sector_per = Sector/len(time_period)
    Type_of_Term_per = Type_of_Term/len(time_period)
    Seniority_per = Seniority/len(time_period)
    Rating_per = Rating/len(time_period)

    Group_sector_per = Group_sector/Group
    Group_Seniority_per = Group_Seniority/Group
    Group_type_term_per = Group_Type_term/Group
    Group_rating_per =  Group_Rating/Group

    Group_Sector_indicator = (Group_sector_per - Sector_per)/(1-Sector_per)
    Group_Seniority_indicator = (Group_Seniority_per - Seniority_per)/(1-Seniority_per)
    Group_type_term_indicator = (Group_type_term_per - Type_of_Term_per)/(1-Type_of_Term_per)
    Group_rating_indicator= (Group_rating_per - Rating_per)/(1-Rating_per)

    for i in range(0,125):
        if i == 0:
            f = Group_Seniority_indicator.loc['Cluster {}'.format(i)].T.head(1).reset_index().iloc[:,1:]
            f['Group'] ='Cluster {}'.format(i)
        else:
            temp = Group_Seniority_indicator.loc['Cluster {}'.format(i)].T.head(1).reset_index().iloc[:,1:]
            temp['Group'] ='Cluster {}'.format(i)
            f = f.merge(temp,how='outer').fillna(0)
    for i in range(0,125):
        if i == 0:
            f1 = Group_Sector_indicator.loc['Cluster {}'.format(i)].T.head(1).reset_index().iloc[:,1:]
            f1['Group'] ='Cluster {}'.format(i)
        else:
            temp1 = Group_Sector_indicator.loc['Cluster {}'.format(i)].T.head(1).reset_index().iloc[:,1:]
            temp1['Group'] ='Cluster {}'.format(i)
            f1 = f1.merge(temp1,how='outer').fillna(0)
    for i in range(0,125):
        if i == 0:
            f2 = Group_type_term_indicator.loc['Cluster {}'.format(i)].T.head(1).reset_index().iloc[:,1:]
            f2['Group'] ='Cluster {}'.format(i)
        else:
            temp2 = Group_type_term_indicator.loc['Cluster {}'.format(i)].T.head(1).reset_index().iloc[:,1:]
            temp2['Group'] ='Cluster {}'.format(i)
            f2 = f2.merge(temp2,how='outer').fillna(0)
    for i in range(0,125):
        if i == 0:
            f3 = Group_rating_indicator.loc['Cluster {}'.format(i)].T.head(1).reset_index().iloc[:,1:]
            f3['Group'] ='Cluster {}'.format(i)
        else:
            temp3 = Group_rating_indicator.loc['Cluster {}'.format(i)].T.head(1).reset_index().iloc[:,1:]
            temp3['Group'] ='Cluster {}'.format(i)
            f3 = f3.merge(temp3,how='outer').fillna(0)
    f= f.merge(f1,on='Group',how = 'left')
    f= f.merge(f2,on='Group',how = 'left')
    f= f.merge(f3,on='Group',how = 'left')
    b = time_period.groupby(by=['Group'],as_index=False).mean()
    b.drop(columns=['SecurityID'],inplace=True)
    b = b.rename(columns={'AmountIssued':'Cluster_average_AmountIssued','Coupon':'Cluster_average_Coupon','Term':'Cluster_average_Term','RatingSP':'Cluster_average_Rating'})
    window_cluster_data = b.merge(f,on='Group',how = 'left')
    return window_cluster_data

In [46]:
from sqlalchemy import create_engine

In [47]:
cluster_info = get_the_cluster_data(security_data,cluster_data,silding_windows[0])

start to get data for 2019-02-28


In [48]:
cluster_info

,Group,Cluster_average_AmountIssued,Cluster_average_Coupon,Cluster_average_Term,Cluster_average_Rating,Jr Subordinated Unsecured,Secured,Sr Unsecured,Subordinated Unsecured,1st Lien Secured,...,Technology,Long Term,Mid Term,Short Term,High grade,Junk,Lower medium grade,UN,Upper medium grade,Prime
0,Cluster 0,6.060661e+08,4.724531,17.824742,16.180732,-0.015001,0.026334,0.167827,0.001681,0.000000,...,0.003382,0.116976,-0.084090,0.016993,0.036947,-0.101761,-0.058181,-0.080262,0.230632,0.000000
1,Cluster 1,9.649194e+08,4.410567,15.305732,16.267418,-0.019042,-0.052738,0.520131,-0.025759,0.003840,...,0.046589,0.155257,-0.018445,-0.045967,0.026352,-0.110199,-0.024581,-0.035896,0.147972,0.005677
2,Cluster 10,7.517519e+08,4.728494,13.470588,14.749780,-0.013508,-0.035366,0.240275,0.009340,0.000813,...,0.046121,0.057325,-0.016825,-0.009606,-0.036570,0.146829,-0.171524,-0.065832,0.138582,0.000000
3,Cluster 100,6.369403e+08,4.613108,15.387097,16.222335,0.000000,0.076778,-0.099423,-0.007304,-0.013469,...,-0.017129,-0.039843,-0.022292,0.031673,0.006940,-0.162829,-0.021329,-0.021094,0.212682,0.003676
4,Cluster 101,6.455059e+08,4.440804,14.774510,16.156443,-0.015519,-0.010113,0.050343,0.019660,-0.005280,...,-0.019101,0.268614,-0.087438,-0.038690,-0.009511,-0.184959,0.113847,-0.005905,0.130299,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
120,Cluster 95,5.961280e+08,5.362838,16.087500,14.619028,0.038524,0.020663,-0.479882,0.039524,0.003098,...,-0.013496,0.171096,-0.026358,-0.046220,-0.021739,0.061699,0.023355,0.044925,-0.108869,0.000000
121,Cluster 96,7.269452e+08,4.357861,12.934307,16.244963,-0.018088,0.005094,0.292956,-0.006478,-0.028183,...,-0.024308,-0.048734,0.011626,0.010136,0.008189,-0.142259,-0.006419,-0.067613,0.249705,0.000000
122,Cluster 97,7.667758e+08,5.426098,17.151515,13.455243,0.028812,-0.008200,-0.182275,0.019191,-0.012203,...,-0.007984,-0.109391,0.105142,-0.035385,-0.021337,0.209649,-0.102571,0.057329,-0.173407,0.000000
123,Cluster 98,6.052778e+08,5.013682,16.270588,15.925284,-0.001443,0.053019,-0.139588,0.009340,0.000000,...,-0.027254,0.057325,-0.067667,0.027786,-0.049058,-0.107625,-0.063383,0.011216,0.221410,0.004695


In [49]:
security_info = security_data[['SecurityID','SecuritySector','AmountIssued','Coupon','Seniority','Term','RatingSP','Rating','Type of Term']]
a = security_info.merge(cluster_data[['SecurityID','2019-02-28']],on=['SecurityID'],how='left')
a.dropna(subset=['2019-02-28'],inplace=True)
time_period = a.rename(columns={'2019-02-28':'Group'})
Group_Seniority = time_period.groupby(by=['Group','Seniority']).count()
Group_sector = time_period.groupby(by=['Group','SecuritySector']).count()
Type_term = time_period.groupby(by=['Group','Type of Term']).count()
Group = time_period.groupby(by=['Group']).count()
Sector = a.groupby(by=['SecuritySector']).count()

In [50]:
Group_sector_per = Group_sector/len(time_period)

In [51]:
cluster_info.to_csv('test1.csv')